In [ ]:
"""Save AWG values for the best data from the DRAG."""

import os
import pickle
from datetime import datetime

import numpy as np
import tensorflow as tf
from c3po.utils import log_setup
from c3po.tf_utils import tf_abs, tf_ave
from c3po.qt_utils import basis, xy_basis, perfect_gate, single_length_RB

from scipy.linalg import expm
import c3po.hamiltonians as hamiltonians
from c3po.simulator import Simulator as Sim
from c3po.optimizer import Optimizer as Opt
from c3po.experiment import Experiment as Exp
from single_qubit import create_chip_model, create_generator, create_gates

# Script parameters
lindbladian = False
IBM_angles = False
search_fid = 'unit'  # 'state' 'unit' 'orbit'
pulse_type = 'drag'  # 'gauss' 'drag' 'pwc'
sim_res = 3e11
awg_res = 2.4e9
awg_sample = 1 / awg_res
sample_numbers = np.arange(4, 20, 1)
logdir = log_setup("/tmp/c3logs/")

# System
qubit_freq = 5.1173e9 * 2 * np.pi
qubit_anhar = -315.513734e6 * 2 * np.pi
qubit_lvls = 4
drive_ham = hamiltonians.x_drive
v_hz_conversion = 1

In [ ]:
base_dir = '/home/usersFWM/froy/Documents/PHD/'
dir_name_drag = 'hockey_open/2019-11-07_13:15/'
savefile_drag = 'drag_N4.pickle'
with open('{}{}{}'.format(base_dir,
                          dir_name_drag, 
                          savefile_drag), 'rb') as file:
    unit_inf = pickle.load(file)
    times = pickle.load(file)
    best_params = pickle.load(file)

In [ ]:
AWG_values = []
for indx in range(len(times)):
    print("#: ", sample_num)
    # Update experiment time
    t_final = times[indx]
    params = best_barams[indx]
    
    # Create system
    model = create_chip_model(qubit_freq, qubit_anhar, qubit_lvls, drive_ham)
    gen = create_generator(sim_res, awg_res, v_hz_conversion, logdir=logdir)
    gen.devices['awg'].options = 'drag'
    gates = create_gates(t_final, v_hz_conversion, qubit_freq, qubit_anhar)
    exp = Exp(model, gen)
    sim = Sim(exp, gates)

    gateset_opt_map = [
        [('X90p', 'd1', 'gauss', 'amp')],
        [('X90p', 'd1', 'gauss', 'freq_offset')],
        [('X90p', 'd1', 'gauss', 'delta')],
        [('X90p', 'd1', 'gauss', 'xy_angle')]
    ]
    
    
    
